# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2233236730>
├── 'a' --> tensor([[-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683]])
└── 'x' --> <FastTreeValue 0x7f2233236070>
    └── 'c' --> tensor([[ 0.0803,  0.0924,  1.0619, -0.5475],
                        [-0.1293, -1.2476,  0.4972,  1.2537],
                        [ 0.0159, -0.3366,  0.3058,  1.6994]])

In [4]:
t.a

tensor([[-0.2546, -0.2132, -0.2158],
        [ 0.1948,  0.4524,  2.3683]])

In [5]:
%timeit t.a

59.3 ns ± 0.0569 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2233236730>
├── 'a' --> tensor([[-0.4512,  1.0961,  0.0724],
│                   [-0.5346, -0.0745, -0.1307]])
└── 'x' --> <FastTreeValue 0x7f2233236070>
    └── 'c' --> tensor([[ 0.0803,  0.0924,  1.0619, -0.5475],
                        [-0.1293, -1.2476,  0.4972,  1.2537],
                        [ 0.0159, -0.3366,  0.3058,  1.6994]])

In [7]:
%timeit t.a = new_value

60.3 ns ± 0.031 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2546, -0.2132, -0.2158],
               [ 0.1948,  0.4524,  2.3683]]),
    x: Batch(
           c: tensor([[ 0.0803,  0.0924,  1.0619, -0.5475],
                      [-0.1293, -1.2476,  0.4972,  1.2537],
                      [ 0.0159, -0.3366,  0.3058,  1.6994]]),
       ),
)

In [10]:
b.a

tensor([[-0.2546, -0.2132, -0.2158],
        [ 0.1948,  0.4524,  2.3683]])

In [11]:
%timeit b.a

59.6 ns ± 0.0108 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1446, -1.1865,  0.3971],
               [-1.8501,  1.3610,  0.2836]]),
    x: Batch(
           c: tensor([[ 0.0803,  0.0924,  1.0619, -0.5475],
                      [-0.1293, -1.2476,  0.4972,  1.2537],
                      [ 0.0159, -0.3366,  0.3058,  1.6994]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.276 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

843 ns ± 0.184 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 17.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 673 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 713 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2183ed6b80>
├── 'a' --> tensor([[[-0.2546, -0.2132, -0.2158],
│                    [ 0.1948,  0.4524,  2.3683]],
│           
│                   [[-0.2546, -0.2132, -0.2158],
│                    [ 0.1948,  0.4524,  2.3683]],
│           
│                   [[-0.2546, -0.2132, -0.2158],
│                    [ 0.1948,  0.4524,  2.3683]],
│           
│                   [[-0.2546, -0.2132, -0.2158],
│                    [ 0.1948,  0.4524,  2.3683]],
│           
│                   [[-0.2546, -0.2132, -0.2158],
│                    [ 0.1948,  0.4524,  2.3683]],
│           
│                   [[-0.2546, -0.2132, -0.2158],
│                    [ 0.1948,  0.4524,  2.3683]],
│           
│                   [[-0.2546, -0.2132, -0.2158],
│                    [ 0.1948,  0.4524,  2.3683]],
│           
│                   [[-0.2546, -0.2132, -0.2158],
│                    [ 0.1948,  0.4524,  2.3683]]])
└── 'x' --> <FastTreeValue 0x7f2183ed6820>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 41.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2183e5bac0>
├── 'a' --> tensor([[-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683],
│                   [-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683],
│                   [-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683],
│                   [-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683],
│                   [-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683],
│                   [-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683],
│                   [-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683],
│                   [-0.2546, -0.2132, -0.2158],
│                   [ 0.1948,  0.4524,  2.3683]])
└── 'x' --> <FastTreeValue 0x7f2183e5baf0>
    └── 'c' --> tensor([[ 0.0803,  0.0924,  1.0619, -0.5475],
                        [-0.1293, -1.2476,  0.4972,  1.2537],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 184 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0803,  0.0924,  1.0619, -0.5475],
                       [-0.1293, -1.2476,  0.4972,  1.2537],
                       [ 0.0159, -0.3366,  0.3058,  1.6994]],
              
                      [[ 0.0803,  0.0924,  1.0619, -0.5475],
                       [-0.1293, -1.2476,  0.4972,  1.2537],
                       [ 0.0159, -0.3366,  0.3058,  1.6994]],
              
                      [[ 0.0803,  0.0924,  1.0619, -0.5475],
                       [-0.1293, -1.2476,  0.4972,  1.2537],
                       [ 0.0159, -0.3366,  0.3058,  1.6994]],
              
                      [[ 0.0803,  0.0924,  1.0619, -0.5475],
                       [-0.1293, -1.2476,  0.4972,  1.2537],
                       [ 0.0159, -0.3366,  0.3058,  1.6994]],
              
                      [[ 0.0803,  0.0924,  1.0619, -0.5475],
                       [-0.1293, -1.2476,  0.4972,  1.2537],
                       [ 0.0159, -0.3366,  0.3058,  1.6994]],

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0803,  0.0924,  1.0619, -0.5475],
                      [-0.1293, -1.2476,  0.4972,  1.2537],
                      [ 0.0159, -0.3366,  0.3058,  1.6994],
                      [ 0.0803,  0.0924,  1.0619, -0.5475],
                      [-0.1293, -1.2476,  0.4972,  1.2537],
                      [ 0.0159, -0.3366,  0.3058,  1.6994],
                      [ 0.0803,  0.0924,  1.0619, -0.5475],
                      [-0.1293, -1.2476,  0.4972,  1.2537],
                      [ 0.0159, -0.3366,  0.3058,  1.6994],
                      [ 0.0803,  0.0924,  1.0619, -0.5475],
                      [-0.1293, -1.2476,  0.4972,  1.2537],
                      [ 0.0159, -0.3366,  0.3058,  1.6994],
                      [ 0.0803,  0.0924,  1.0619, -0.5475],
                      [-0.1293, -1.2476,  0.4972,  1.2537],
                      [ 0.0159, -0.3366,  0.3058,  1.6994],
                      [ 0.0803,  0.0924,  1.0619, -0.5475],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 425 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 7.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
